In [9]:
from dotenv import load_dotenv
import os
import re
from PyPDF2 import PdfReader
from tqdm import tqdm
from langchain.docstore.document import Document
from langchain.chains.summarize import load_summarize_chain
from langchain.vectorstores import FAISS
from langchain.schema import Document

In [10]:
from langchain_ollama import ChatOllama

In [11]:
llm=ChatOllama(model='gemma3:1b')

In [12]:
file_path='harry1.pdf'

In [13]:
from PyPDF2 import PdfReader
reader = PdfReader("harry1.pdf")
full_text=' '.join( [page.extract_text()for page in reader.pages] )

In [14]:
chapter_sections = re.split(r'(CHAPTER\s[A-Z]+(?:\s[A-Z]+)*)', full_text)

In [15]:
chapter_sections[1],chapter_sections[2]

('CHAPTER\tONE',
 "\n\t\nTHE\tBOY\tWHO\tLIVED\n\t\n\t\t\t\t\t\t\nM\t\nr.\tand\tMrs.\tDursley,\tof\tnumber\tfour,\tPrivet\tDrive,\twere\tproud\tto\tsay\nthat\tthey\twere\tperfectly\tnormal,\tthank\tyou\tvery\tmuch.\tThey\twere\tthe\tlast\tpeople\nyou’d\texpect\tto\tbe\tinvolved\tin\tanything\tstrange\tor\tmysterious,\tbecause\tthey\tjust\ndidn’t\thold\twith\tsuch\tnonsense.\n\t\t\t\t\t\tMr.\tDursley\twas\tthe\tdirector\tof\ta\tfirm\tcalled\tGrunnings,\twhich\tmade\ndrills.\tHe\twas\ta\tbig,\tbeefy\tman\twith\thardly\tany\tneck,\talthough\the\tdid\thave\ta\nvery\tlarge\tmustache.\tMrs.\tDursley\twas\tthin\tand\tblonde\tand\thad\tnearly\ttwice\tthe\nusual\tamount\tof\tneck,\twhich\tcame\tin\tvery\tuseful\tas\tshe\tspent\tso\tmuch\tof\ther\ntime\tcraning\tover\tgarden\tfences,\tspying\ton\tthe\tneighbors.\tThe\tDursleys\thad\ta\nsmall\tson\tcalled\tDudley\tand\tin\ttheir\topinion\tthere\twas\tno\tfiner\tboy\tanywhere.\n\t\t\t\t\t\tThe\tDursleys\thad\teverything\tthey\twanted,\tbut\tthey\ta

In [16]:
chapters = []
for i in range(1,len(chapter_sections),2):
    chapter_text=chapter_sections[i]+chapter_sections[i+1]
    doc = Document(page_content=chapter_text, metadata={"chapter": i // 2 + 1})
    chapters.append(doc)

In [9]:
chapters

[Document(metadata={'chapter': 1}, page_content="CHAPTER\tONE\n\t\nTHE\tBOY\tWHO\tLIVED\n\t\n\t\t\t\t\t\t\nM\t\nr.\tand\tMrs.\tDursley,\tof\tnumber\tfour,\tPrivet\tDrive,\twere\tproud\tto\tsay\nthat\tthey\twere\tperfectly\tnormal,\tthank\tyou\tvery\tmuch.\tThey\twere\tthe\tlast\tpeople\nyou’d\texpect\tto\tbe\tinvolved\tin\tanything\tstrange\tor\tmysterious,\tbecause\tthey\tjust\ndidn’t\thold\twith\tsuch\tnonsense.\n\t\t\t\t\t\tMr.\tDursley\twas\tthe\tdirector\tof\ta\tfirm\tcalled\tGrunnings,\twhich\tmade\ndrills.\tHe\twas\ta\tbig,\tbeefy\tman\twith\thardly\tany\tneck,\talthough\the\tdid\thave\ta\nvery\tlarge\tmustache.\tMrs.\tDursley\twas\tthin\tand\tblonde\tand\thad\tnearly\ttwice\tthe\nusual\tamount\tof\tneck,\twhich\tcame\tin\tvery\tuseful\tas\tshe\tspent\tso\tmuch\tof\ther\ntime\tcraning\tover\tgarden\tfences,\tspying\ton\tthe\tneighbors.\tThe\tDursleys\thad\ta\nsmall\tson\tcalled\tDudley\tand\tin\ttheir\topinion\tthere\twas\tno\tfiner\tboy\tanywhere.\n\t\t\t\t\t\tThe\tDursleys\tha

In [10]:
print(f"Total number of chapters extracted: {len(chapters)}")

Total number of chapters extracted: 17


In [17]:
book_quotes_list = []
min_length = 50

In [18]:

import re
from tqdm import tqdm
from langchain.schema import Document  # Assuming you're using LangChain

# Set minimum quote length
min_length = 20

# Compile regex to match quotes longer than min_length
quote_pattern_longer_than_min_length = re.compile(
    rf'"(.{{{min_length},}}?)"', re.DOTALL
)

# Initialize an empty list to store the quote documents
book_quotes_list = []

# Iterate through each chapter document to find and extract quotes
for doc in tqdm(chapters, desc="Extracting quotes"):
    content = doc.page_content
    found_quotes = quote_pattern_longer_than_min_length.findall(content)

    for quote in found_quotes:
        quote_doc = Document(
            page_content=quote,
            metadata={"chapter": doc.metadata.get("chapter", "unknown")}
        )
        book_quotes_list.append(quote_doc)


Extracting quotes: 100%|██████████| 17/17 [00:00<00:00, 26896.71it/s]


In [13]:
print(book_quotes_list)

[]


In [14]:
import re
from langchain.schema import Document

def extract_book_quotes_as_documents(documents, min_length=50):
    """
    Extracts quotes longer than `min_length` characters from a list of Document objects.

    Args:
        documents (list): List of LangChain Document objects.
        min_length (int): Minimum quote length to extract.

    Returns:
        list: List of Document objects, each containing an extracted quote.
    """
    quotes_as_documents = []
    quote_pattern = re.compile(rf'"(.{{{min_length},}}?)"', re.DOTALL)

    for doc in documents:
        content = doc.page_content.replace('\n', ' ')
        found_quotes = quote_pattern.findall(content)

        for quote in found_quotes:
            quote_doc = Document(
                page_content=quote.strip(),
                metadata=doc.metadata  # Preserves chapter info or other metadata
            )
            quotes_as_documents.append(quote_doc)

    return quotes_as_documents


In [15]:
book_quotes_list = extract_book_quotes_as_documents(chapters, min_length=50)


In [16]:
book_quotes_list

[]

In [17]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [18]:
chunk_size = 1000 
chunk_overlap = 200

In [19]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=chunk_size, chunk_overlap=chunk_overlap, length_function=len
)

document_splits = text_splitter.split_documents(chapters)

In [20]:
print(f"Number of documents after splitting: {len(document_splits)}")

Number of documents after splitting: 571


In [21]:
print(f"First chapter content: {chapters[0].page_content[:500]}...")

First chapter content: CHAPTER	ONE
	
THE	BOY	WHO	LIVED
	
						
M	
r.	and	Mrs.	Dursley,	of	number	four,	Privet	Drive,	were	proud	to	say
that	they	were	perfectly	normal,	thank	you	very	much.	They	were	the	last	people
you’d	expect	to	be	involved	in	anything	strange	or	mysterious,	because	they	just
didn’t	hold	with	such	nonsense.
						Mr.	Dursley	was	the	director	of	a	firm	called	Grunnings,	which	made
drills.	He	was	a	big,	beefy	man	with	hardly	any	neck,	although	he	did	have	a
very	large	mustache.	Mrs.	Dursley	was	thin	...


In [22]:
tab_pattern = re.compile(r'\t')

for doc in chapters:
    doc.page_content = tab_pattern.sub(' ', doc.page_content)

In [23]:
print(f"First cleaned chapter content: {chapters[0].page_content[:500]}...")

First cleaned chapter content: CHAPTER ONE
 
THE BOY WHO LIVED
 
      
M 
r. and Mrs. Dursley, of number four, Privet Drive, were proud to say
that they were perfectly normal, thank you very much. They were the last people
you’d expect to be involved in anything strange or mysterious, because they just
didn’t hold with such nonsense.
      Mr. Dursley was the director of a firm called Grunnings, which made
drills. He was a big, beefy man with hardly any neck, although he did have a
very large mustache. Mrs. Dursley was thin ...


In [24]:
multiple_newlines_pattern = re.compile(r'\n\s*\n')

In [25]:
word_split_newline_pattern = re.compile(r'(\w)\n(\w)')

In [26]:
multiple_spaces_pattern = re.compile(r' +')

In [27]:
for doc in chapters:
    # 1. Replace multiple newlines with a single newline.
    page_content = multiple_newlines_pattern.sub('\n', doc.page_content)

    # 2. Remove newlines that are not followed by a space or another newline.
    page_content = word_split_newline_pattern.sub(r'\1\2', doc.page_content)

    # 3. Replace any remaining single newlines (often within paragraphs) with a space.
    page_content = page_content.replace('\n', ' ')

    # 4. Reduce multiple spaces to a single space.
    page_content = multiple_spaces_pattern.sub(' ', page_content)
    
    doc.page_content = page_content

In [28]:
print(f"First cleaned chapter content: {chapters[15].page_content[:500]}...")


First cleaned chapter content: CHAPTER SIXTEEN THROUGH THE TRAPDOOR I n years to come, Harry would never quite remember how he had managedto get through his exams when he half expected Voldemort to come burstingthrough the door at any moment. Yet the days crept by, and there could be nodoubt that Fluffy was still alive and well behind the locked door. It was sweltering hot, especially in the large classroom where they didtheir written papers. They had been given special, new quills for the exams, which had been bewitched with...


In [29]:
for doc in document_splits:
    # Replace tab characters with a single space
    doc.page_content = tab_pattern.sub(' ', doc.page_content)
    
    # Collapse multiple newlines into a single newline
    doc.page_content = multiple_newlines_pattern.sub('\n', doc.page_content)
    
    # Fix word splits across newlines (e.g., "mag-\nic" -> "magic")
    doc.page_content = word_split_newline_pattern.sub(r'\1\2', doc.page_content)
    
    # Collapse multiple spaces into a single space
    doc.page_content = multiple_spaces_pattern.sub(' ', doc.page_content)

In [30]:
# Calculate the word count for each chapter by splitting the page_content on whitespace
chapter_word_counts = [len(doc.page_content.split()) for doc in chapters]

# Find the maximum number of words in a chapter
max_words = max(chapter_word_counts)

# Find the minimum number of words in a chapter
min_words = min(chapter_word_counts)

# Calculate the average number of words per chapter
average_words = sum(chapter_word_counts) / len(chapter_word_counts)

# Print the statistics
print(f"Max words in a chapter: {max_words}")
print(f"Min words in a chapter: {min_words}")
print(f"Average words in a chapter: {average_words:.2f}")


Max words in a chapter: 6348
Min words in a chapter: 2918
Average words in a chapter: 4404.94


In [31]:
from langchain.prompts import PromptTemplate

template = """Write an extensive summary of the following:

{text}

SUMMARY:"""

# Initialize the PromptTemplate with the template and input variables
# The template expects one input variable called "text"

summarization_prompt = PromptTemplate(
    template=template,
    input_variables=["text"]
)

In [32]:
from tqdm import tqdm  # Add this import

chain = load_summarize_chain(llm, chain_type="stuff", prompt=summarization_prompt)

# Initialize a list to store the summaries
chapter_summaries = []

# Use tqdm to wrap the iterable
for chapter in tqdm(chapters, desc="Summarizing chapters"):
    # Generate summary using the chain
    summary = chain.invoke([chapter])

    # Clean the output text
    cleaned_text = re.sub(r'\n\n', '\n', summary["output_text"])

    # Create a Document object for the summary, preserving the original metadata
    doc_summary = Document(page_content=cleaned_text, metadata=chapter.metadata)
    chapter_summaries.append(doc_summary)


Summarizing chapters: 100%|██████████| 17/17 [52:39<00:00, 185.86s/it]


In [33]:
import json

# Serialize summaries to JSON
with open("chapter_summaries.json", "w", encoding="utf-8") as f:
    json.dump([
        {"page_content": doc.page_content, "metadata": doc.metadata}
        for doc in chapter_summaries
    ], f, ensure_ascii=False, indent=2)


In [ ]:


with open("chapter_summaries.json", "r", encoding="utf-8") as f:
    loaded_data = json.load(f)

chapter_summaries = [
    Document(page_content=item["page_content"], metadata=item["metadata"])
    for item in loaded_data
]


In [5]:
from langchain_ollama import OllamaEmbeddings

embed_llm = OllamaEmbeddings(model="mxbai-embed-large")


In [8]:
# Create a FAISS vector store from the document splits using the embedding model
book_splits_vectorstore = FAISS.from_documents(document_splits, embed_llm)

# Create a FAISS vector store from the chapter summaries using the embedding model
chapter_summaries_vectorstore = FAISS.from_documents(chapter_summaries, embed_llm)

# Create a FAISS vector store from the quotes using the embedding model
#quotes_vectorstore = FAISS.from_documents(book_quotes_list, embed_llm)

NameError: name 'document_splits' is not defined

In [ ]:
#quotes_vectorstore.save_local("quotes_vectorstore")

In [ ]:
# quotes_vectorstore = FAISS.load_local(
#     "quotes_vectorstore",           # Path to the saved FAISS index for quotes
#     embed_llm,                # Embedding model used for encoding queries and documents
#     allow_dangerous_deserialization=True  # Allows loading objects that may not be fully secure (required for FAISS)
# )

In [ ]:
# Retriever for book content chunks (splits), returns the top 1 most relevant chunk.
book_chunks_retriever = book_splits_vectorstore.as_retriever(search_kwargs={"k": 1})

# Retriever for chapter summaries, returns the top 1 most relevant summary.
chapter_summaries_retriever = chapter_summaries_vectorstore.as_retriever(search_kwargs={"k": 1})

# Retriever for book quotes, returns the top 10 most relevant quotes.
#book_quotes_retriever = quotes_vectorstore.as_retriever(search_kwargs={"k": 10})